In [ ]:
import requests
import time
import json
import base64
import re
import os
from dotenv import load_dotenv

load_dotenv()

# ================= CONFIGURAZIONE =================
# INSERISCI QUI LA TUA API KEY DI VIRUSTOTAL
VT_API_KEY = os.getenv("VIRUS_API_KEY")

OUTPUT_FILE = "virustotal_results.json"

# La lista dei 30 IoC selezionati nel passaggio precedente
TARGET_IOCS = [
    # IPs e Domini ad alta probabilità 
    '51.142.173.103',
    '51.143.138.171',
    '146.59.102.109',
    '141.98.118.213',
    '45.89.52.197',
    '209.38.160.45',
    '51.143.138.181',
    'trojan.petr.cc',
    '62.133.61.35',
    '62.113.115.180',
    '51.143.138.95',
    '51.143.138.174',
    '3.70.23.209',
    # Stringhe Shadowsocks (vengono trattate come URL complessi)
    'ss://Y2hhY2hhMjAtaWV0Zi1wb2x5MTMwNToxZjhqZE9JS2t4UldNVWVPZlhQZW84:None@109.107.172.30:28190',
    'ss://Y2hhY2hhMjAtaWV0Zi1wb2x5MTMwNTpnY3BWZWhWeXRzbGVwbWtISFJMWFl4:None@51.142.173.103:51899',
    'https://doxbin.com/upload/DOXXING20GUIDE',
    'https://api.neostress.net/stop?token=',
    # HASHES
    '050f9e667e41ec8663b2709db6e0d4e08eba9b1d734e2389c5c3afdfb5a4231e',
    '754FA0CFF33F2F59F4B8CA3DEB4C2CDA',
    'B33FAB5C07EF12398A540C7757F6E065',
    '27559c37e0cd3ac8ec4df3238094c850be194821aa9060098ea0e42c5d6d7332',
    '252e83bfa7e739167b3118bbe1c9fbd7636e33bf7cf85d0f17c7c02b8f549c41',
    'AB4F368345DC20DC59516C098AAD331C',
    'B945A8AD3DDFF937B939CDB6E39344A5',
    '6C381BEE9E3B57CA5BF0104DDD22077E',
    '64fcb13849fe1227797baad003500b8a2adf2140c128d9bd8ab9a2377287b6ba',
    '96459b62e688ea4bff11252924c053f9d0b23fb80ffdfc7c0cfa3cd9b6a5db67',
    '1B0D4EE4D3E20B8922251941B956C21E',
    'D1B822529EB33B316FF2C926DFF30D12',
    '50EB49BA3CDD036447E7C9E641F34A6D',
    '70DF9465BA2AA8815B211BBB27F6F742',
    'CEE271F60FA1BAA8AF316333E91FEE56',
    # URLs / DOMAINS
    'https://doxbin.com/upload/DOXXING20GUIDE',
    'https://api.neostress.net/stop?token=',
    'trojan.petr.cc',
    'trojan.tr',
    'https://happytiger.co.uk',
    'https://blogcay.com/',
    'https://byron.ro/en/blog/',
    'https://my.centorly.ru/login',
    'my.centorly.ru',
    'generic.mg',
    'mail.alfa.ru',
    'https://hit.club',
    'https://vpswala.org',
    'https://www.nic.ru/whois/?searchWord=172.25.71.42',
    'https://blogcay.com',
    'https://www.hetzner.com',  # Spesso segnalato per hosting abuse
    'trojan.petr.cc'
]

# ================= FUNZIONI =================

def get_ioc_type(ioc):
    """Determina il tipo di IoC per l'API v3"""
    # IP Address
    if re.match(r"^\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}$", ioc):
        return "ip_addresses"
    # Hash (MD5, SHA1, SHA256)
    elif re.match(r"^[a-fA-F0-9]{32}$", ioc) or re.match(r"^[a-fA-F0-9]{40}$", ioc) or re.match(r"^[a-fA-F0-9]{64}$", ioc):
        return "files"
    # URL (inizia con http/https)
    elif ioc.startswith("http://") or ioc.startswith("https://"):
        return "urls"
    # Dominio (tutto il resto)
    else:
        return "domains"

def check_virustotal(ioc):
    ioc_type = get_ioc_type(ioc)
    endpoint = ioc
    
    # Gestione speciale per gli URL (Base64 encoding richiesto da VT)
    if ioc_type == "urls":
        endpoint = base64.urlsafe_b64encode(ioc.encode()).decode().strip("=")
    
    url = f"https://www.virustotal.com/api/v3/{ioc_type}/{endpoint}"
    headers = {
        "x-apikey": VT_API_KEY,
        "accept": "application/json"
    }
    
    try:
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            data = response.json()
            attr = data['data']['attributes']
            stats = attr['last_analysis_stats']
            
            # Estraiamo dati utili
            return {
                "ioc": ioc,
                "type": ioc_type,
                "malicious": stats['malicious'],
                "suspicious": stats['suspicious'],
                "harmless": stats['harmless'],
                "undetected": stats['undetected'],
                "total_engines": sum(stats.values()),
                "link": data['data']['links']['self']
            }
            
        elif response.status_code == 404:
            print(f"   ❌ {ioc} -> Non trovato su VT.")
            return None
        elif response.status_code == 429:
            print("   ⏳ Quota superata! Attendo 60 secondi extra...")
            time.sleep(60)
            return check_virustotal(ioc) # Riprova
        else:
            print(f"   ⚠️ Errore {response.status_code}: {response.text}")
            return None
            
    except Exception as e:
        print(f"   ⚠️ Eccezione: {e}")
        return None

# ================= MAIN =================

print(f"🚀 Inizio scansione di {len(TARGET_IOCS)} IoC su VirusTotal...")
# Puliamo il file all'avvio per evitare di accodare ai vecchi test
if os.path.exists(OUTPUT_FILE):
    os.remove(OUTPUT_FILE)

for i, ioc in enumerate(TARGET_IOCS):
    print(f"[{i+1}/{len(TARGET_IOCS)}] Checking: {ioc} ...", end=" ", flush=True)
    
    result = check_virustotal(ioc)
    
    if result:
        # Aggiungiamo un timestamp per Wazuh (opzionale ma consigliato)
        result['timestamp'] = time.strftime("%Y-%m-%dT%H:%M:%S%z")

        score = result['malicious']
        total = result['total_engines']
        
        if score > 0:
            print(f"🚨 MALICIOUS ({score}/{total})")
        else:
            print(f"✅ CLEAN ({score}/{total})")
            
        # --- MODIFICA FONDAMENTALE PER WAZUH ---
        # Apriamo in modalità 'a' (append) e scriviamo su una sola riga
        with open(OUTPUT_FILE, 'a') as f:
            f.write(json.dumps(result) + "\n")
        # ---------------------------------------

    else:
        print("") 

    # Sleep mandatorio per account free
    if i < len(TARGET_IOCS) - 1:
        time.sleep(15)

print(f"\n🏁 Finito! Log pronti per Wazuh in: {OUTPUT_FILE}")